In [1]:
import numpy as np
import time
from sorted_row_collisions import *
from gpu_attempt_test import *
#from hash import *

In [2]:
def get_decimal_edge_distance(adjacency_matrix):
    return np.sum(np.matmul(
        adjacency_matrix,
        2 ** get_edge_distance_matrix(adjacency_matrix.shape[0], adjacency_matrix.shape[1])),
        axis=1)

In [3]:
def get_edge_distance_matrix(height, width):
    arr_1 = np.arange(width)
    arr_2 = np.arange(height).reshape(height, 1)

    repeated_arr_1 = np.tile(arr_1, (height, 1))
    repeated_arr_2 = np.tile(arr_2, (1, width))

    return np.minimum(repeated_arr_1, repeated_arr_2)

In [4]:
def get_score(adjacency_matrix):
    row_sum = np.sum(adjacency_matrix, axis=0)
    col_score = np.matmul(row_sum, adjacency_matrix.T)
    return col_score

In [5]:
#the matrix is getting transformed in a way that shouldn't be allowed. There is a different operation than the rows and columns just being swapped
def full_hash_1(matrix):
    matrix, row_diags, row_order_1 = matrix_row_collisions_split_then_sorted_1(matrix)
    print("matrix after rows sorted by diagonals")
    print(matrix)
    matrix, col_diags, col_order_1 = matrix_row_collisions_split_then_sorted_1(matrix.T)
    matrix = matrix.T
    print("matrix after cols sorted by diagonals")
    print(matrix)

    #sort matrix rows
    decimal_matrix = get_row_bit_scores(matrix)
    print(f"row diags: {row_diags.reshape(-1)} row order: {row_order_1.reshape(-1)} decimal: {decimal_matrix.reshape(-1)}")

    row_collisions = np.concatenate([get_score(matrix).reshape(-1,1), row_diags, row_order_1, decimal_matrix],axis=1)
    argsorted_rows_indecis = order_of_rows(row_collisions)

    matrix = matrix[argsorted_rows_indecis]

    #sort matrix cols
    decimal_matrix = get_row_bit_scores(matrix.T)
    print(f"col diags: {col_diags.reshape(-1)} col order: {col_order_1.reshape(-1)} decimal: {decimal_matrix.reshape(-1)}")

    col_collisions = np.concatenate([get_score(matrix.T).reshape(-1,1), col_diags, col_order_1, decimal_matrix],axis=1)
    argsorted_col_indecis = order_of_rows(col_collisions)

    #matrix = matrix.T[argsorted_col_indecis]
    matrix = matrix[:, argsorted_col_indecis]

    return matrix

In [6]:
#maybe sort by decimal last?
def matrix_row_collisions_split_then_sorted_1(matrix):
    matrix, row_collisions = get_matrix_and_row_collisions_sorted_by_row_collision_diagonals(matrix)

    diags = np.sum(row_collisions, axis=1)

    #get row collision order
    row_collisions = sort_chunked_row_collisions(row_collisions)
    #row_collision_order = custom_row_order(np.fliplr(row_collisions)).reshape(-1,1)
    row_collision_order = order_of_rows(row_collisions).reshape(-1,1) #since this gets sorted, this should be fine to order???

    return matrix, diags.reshape(-1,1), row_collision_order

In [17]:
def new_hash(matrix):
    matrix, row_collisions, col_collisions = matrix_row_and_col_collisions(matrix)

    #row sort starts
    row_collision_diags = np.diag(row_collisions).reshape(-1,1)
    decimal_matrix = get_row_bit_scores(matrix).reshape(-1,1)

    row_sorting_matrix = np.concatenate([row_collision_diags, row_collisions, decimal_matrix],axis=1)
    argsorted_rows_indecis = new_order_of_rows(row_sorting_matrix)
    matrix = matrix[argsorted_rows_indecis]

    #col sort starts
    col_collision_diags = np.diag(col_collisions).reshape(-1,1)
    decimal_matrix = get_row_bit_scores(matrix.T).reshape(-1,1)

    row_sorting_matrix = np.concatenate([col_collision_diags, col_collisions, decimal_matrix],axis=1)
    argsorted_rows_indecis = new_order_of_rows(row_sorting_matrix)
    matrix = matrix.T[argsorted_rows_indecis]

    return matrix.T

In [80]:
def get_order_of_chunked_row_collisions(collision_matrix):
    chunked_row_collisions = sort_chunked_row_collisions(collision_matrix)
    chunked_row_order = new_order_of_rows(chunked_row_collisions)#.reshape(-1,1) #this is still wrong...
    #print(f"chunked collisions:")
    #print(chunked_row_collisions)
    return chunked_row_order

In [128]:
def mini_hash(matrix):
    matrix, row_collisions, col_collisions = matrix_row_and_col_collisions(matrix)

    #print("-------------matrix-----------")
    #print(matrix)
    #print("---------row collisions-----------")
    #print(row_collisions)
    #print("---------col collisions-----------")
    #print(col_collisions)


    #start row sorts
    chunked_row_order = get_order_of_chunked_row_collisions(row_collisions).reshape(-1,1)
    decimal_matrix = get_row_bit_scores(matrix).reshape(-1,1)

    row_sorting_matrix = np.concatenate([chunked_row_order, decimal_matrix],axis=1)
    #print(f"row sorting matrix:")
    #print(row_sorting_matrix)
    argsorted_rows_indecis = new_order_of_rows(np.fliplr(row_sorting_matrix))#order_of_rows(row_sorting_matrix)

    #print(f"arg sorted rows: {argsorted_rows_indecis.reshape(-1)}")

    matrix = matrix[argsorted_rows_indecis]
    #print("matrix after row sorts")
    #print(matrix)

    #col sort starts
    chunked_col_order = get_order_of_chunked_row_collisions(col_collisions).reshape(-1,1)
    decimal_matrix = get_row_bit_scores(matrix.T).reshape(-1,1)

    col_sorting_matrix = np.concatenate([chunked_col_order, decimal_matrix],axis=1)
    #print(f"col sorting matrix:")
    #print(col_sorting_matrix)
    argsorted_cols_indecis = new_order_of_rows(np.fliplr(col_sorting_matrix))#order_of_rows(row_sorting_matrix)
    #print(f"arg sorted cols: {argsorted_cols_indecis.reshape(-1)}")
    matrix = matrix[:, argsorted_cols_indecis]

    #print("matrix after col sorts")
    #print(matrix)
    #print("--------------------")
    #print(row_collisions)
    #print("--------------------")
    #print(col_collisions)

    return matrix

In [129]:
def mini_hash_2_1(matrix):
    matrix, row_collisions, col_collisions = matrix_row_and_col_collisions(matrix)

    #start row sorts
    chunked_row_order = get_order_of_chunked_row_collisions(row_collisions).reshape(-1,1)
    decimal_matrix = get_row_bit_scores(matrix).reshape(-1,1)

    row_sorting_matrix = np.concatenate([decimal_matrix, chunked_row_order],axis=1)
    argsorted_rows_indecis = new_order_of_rows(row_sorting_matrix)

    matrix = matrix[argsorted_rows_indecis]

    #col sort starts
    chunked_col_order = get_order_of_chunked_row_collisions(col_collisions).reshape(-1,1)
    decimal_matrix = get_row_bit_scores(matrix.T).reshape(-1,1)

    col_sorting_matrix = np.concatenate([decimal_matrix, chunked_col_order],axis=1)
    argsorted_cols_indecis = new_order_of_rows(col_sorting_matrix)
    matrix = matrix[:, argsorted_cols_indecis]

    return matrix

In [130]:
def mini_hash_2(matrix):
    return mini_hash(mini_hash(matrix))

def mini_hash_2_2(matrix):
    return mini_hash_2_1(mini_hash_2_1(matrix))

In [143]:
def mini_hash_3(matrix):
    matrix, row_collisions, col_collisions = matrix_row_and_col_collisions(matrix)

    #start row sorts
    chunked_row_order = np.argsort(get_order_of_chunked_row_collisions(row_collisions))
    chunked_col_order = np.argsort(get_order_of_chunked_row_collisions(col_collisions))
    matrix = matrix[chunked_row_order][:, chunked_col_order]

    #decimal row sorting
    decimal_matrix = get_row_bit_scores(matrix).reshape(-1,1)
    row_sorting_matrix = np.concatenate([decimal_matrix, chunked_row_order.reshape(-1,1)],axis=1)
    argsorted_rows_indecis = new_order_of_rows(row_sorting_matrix)
    matrix = matrix[argsorted_rows_indecis]

    #decimal col sorting
    decimal_matrix = get_row_bit_scores(matrix.T).reshape(-1,1)
    col_sorting_matrix = np.concatenate([decimal_matrix, chunked_col_order.reshape(-1,1)],axis=1)
    argsorted_cols_indecis = new_order_of_rows(col_sorting_matrix)
    matrix = matrix[:, argsorted_cols_indecis]

    return matrix

In [144]:
a = format_array('[[0 0 0 1] [1 0 0 1] [0 1 1 0] [0 0 0 0]]')
b = format_array('[[0 0 0 0] [0 0 0 1] [1 1 0 0] [0 0 1 1]]')
c = format_array('[[0 0 0 1] [0 1 1 0] [1 0 0 1] [0 0 0 0]]')

In [145]:
a = format_array('[[0 0 0 0] [0 0 1 0] [1 0 0 0] [0 1 0 1]]')
b = format_array('[[0 0 0 0] [1 0 0 0] [0 0 1 0] [0 1 0 1]]')
c = b.T

In [141]:
print(a)
matrix = mini_hash(a)
matrix

[[0 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 1]]


array([[0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 1]])

In [119]:
print(c)
matrix = mini_hash_3(a)
matrix

[[0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]]


array([[0, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 1]])

In [109]:
matrix = mini_hash_3(mini_hash_3(a))
matrix

array([[0, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 1]])

In [135]:
fast_hash_testing(matrix, mini_hash_2_2)

In [146]:
test_matrix(mini_hash_2_2)

num of matrices: 65536
0
[[0 0 0 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 1]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [0 0 1 0]]


AssertionError: 

In [97]:
test_matrix(mini_hash_2)

num of matrices: 65536
0
1000
2000


KeyboardInterrupt: 

In [86]:
test_matrix(new_hash)

num of matrices: 65536
0
[[0 0 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 0]]
[[0 0 0 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]]


AssertionError: 

In [ ]:
def new_order_of_rows(matrix):
    return 0

In [32]:
collision_matrix = np.array([[0, 0, 1, 1],
                   [0, 1, 1, 0],
                   [0, 1, 1, 0],
                   [0, 0, 1, 2]])
expected_array = [1, 1, 0, 2]

In [43]:
collision_matrix

array([[0, 0, 1, 1],
       [0, 1, 1, 0],
       [0, 1, 1, 0],
       [0, 0, 1, 2]])

In [44]:
collision_matrix.T

array([[0, 0, 0, 0],
       [0, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 0, 0, 2]])

In [54]:
def new_order_of_rows(collision_matrix):
    prev_order = np.lexsort(collision_matrix.T)
    #print(f"previous index: {prev_order}")
    prev_row = collision_matrix[prev_order][0]
    prev_index = 0
    for index, row in enumerate(collision_matrix[prev_order]):
        #print(row)
        if not np.allclose(prev_row, row):
            prev_index = index
            prev_row = row
        prev_order[index] = prev_order[prev_index]
    return prev_order

previous index: [1 2 0 3]
[0 1 1 0]
[0 1 1 0]
[0 0 1 1]
[0 0 1 2]


array([1, 1, 0, 3], dtype=int64)